# Lesson3 系列データで分類・予測させてみよう（RNN, LSTM）

## 目次

- Section1 Checkクイズの解答
- Section2. 実装①
    - 2.0 データの用意
    - 2.1 モデル構築
    - 2.2 モデルの学習
    - 2.3 モデルによる予測
    - 2.4 モデルの可視化

## Section1 Checkクイズの解答

問題1: 2, 問題2: 3

## Section2 実装①

ここでは、以下のページのECGによる波形データセットECG5000をRNNで学習し、分類を行ってみましょう。

http://timeseriesclassification.com/description.php?Dataset=ECG5000

このデータセットは、ECGの計測値140時点分とその系列に対する分類（1～5）で1データとなっており、全体で5000データが含まれています。

### 2.0 データの用意

**ここで利用するデータセットはリポジトリに含まれていません。**

上記リンク先のページ中段にある "Download this dataset" をクリックの上、ご自身でダウンロードして頂くようお願い致します。

ダウンロードしたデータセットはdataフォルダ内に配置していただく必要があります。

データセットの配置が完了したら、まずは配置したデータセットの読み込みを行いましょう。

arffという形式で保存されているため、`scipy.io.loadarff`関数でロードします。

https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.arff.loadarff.html

In [ ]:
%matplotlib inline

from scipy.io import arff
import numpy as np

dataset, meta = arff.loadarff('data/ECG5000.arff')

ds = np.asarray(dataset.tolist(), dtype=np.float32)
x_dataset = ds[:, :140]
y_dataset = np.asarray(ds[:,-1].tolist(), dtype=np.int8)-1

実際に表示してみて、様子を確認しておきましょう。

In [ ]:
import matplotlib.pyplot as plt

N = 10
print(y_dataset[:N])
obj = plt.plot(x_dataset[:N].T)
plt.legend(obj, [str(n) for n in range(N)])

確かに、分類結果が異なる3,7番はt=100付近でピークを持たないことが見て取れます。

続いて、データセットを訓練用とテスト用に分割します。

In [ ]:
# 訓練データとテストデータを分割（＋データの整形）

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

x_train, x_test, y_train, y_test = train_test_split(x_dataset[:,:,np.newaxis], to_categorical(y_dataset), test_size=0.2, random_state=42)

### 2.1 モデル構築

ここでは、先程紹介したSimpleRNNを使用してモデルを構築します。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN

hid_dim = 10

# SimpleRNNにDenseを接続し、分類
model = Sequential()

model.add(SimpleRNN(hid_dim, input_shape=x_train.shape[1:])) # input_shape=(系列長T, x_tの次元), output_shape=(units(=hid_dim),)
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 2.2 モデルの学習


In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=100, verbose=2, validation_split=0.2)

### 2.3 モデルによる分類精度の評価

In [ ]:
# 性能評価
score = model.evaluate(x_test, y_test, verbose=0)
print('test_loss:', score[0])
print('test_acc:', score[1])

### 2.4 モデルの可視化

In [ ]:
from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))